# Inherited from 12-3
Add add `MyRNN` class

<img src='ex12-5.png'>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(99999999)

In [11]:
full_data = 'hihello'
full_data = set([i for i in full_data])
full_data # Try minor change in arrangement

{'e', 'h', 'i', 'l', 'o'}

In [12]:
word_to_ix = {'e': 0, 'h': 1, 'i': 2, 'l': 3, 'o': 4}
embeds = nn.Embedding(5, 5) # 5 chars, 5 dimensions

x_data = 'hihell'

lookup_tensor = torch.tensor([word_to_ix[i] for i in x_data], dtype=torch.long)
inputs = embeds(lookup_tensor)

In [13]:
lookup_tensor

tensor([1, 2, 1, 0, 3, 3])

In [14]:
# Need to do this. Otherwise `AutoGrad` will be `Embedding` not mine
inputs = inputs.clone().detach()
inputs

tensor([[-1.0195,  0.2725,  0.2672, -2.0831, -0.9972],
        [ 0.9105, -0.7068,  1.2200, -0.0353,  0.5045],
        [-1.0195,  0.2725,  0.2672, -2.0831, -0.9972],
        [ 0.2432, -0.0211,  0.7164, -0.5276,  0.0104],
        [-0.9600, -0.7599,  0.5004, -0.4419,  0.2254],
        [-0.9600, -0.7599,  0.5004, -0.4419,  0.2254]])

In [15]:
tmp = [word_to_ix[i] for i in 'ihello']
labels = torch.tensor(tmp, dtype=torch.long)

In [16]:
labels

tensor([2, 1, 0, 3, 3, 4])

# (2) Parameters

In [17]:
num_classes = 5
input_size = 5  # one_hot size
hidden_size = 5 # output from the LSTM. 5 to directly predict one-hot
batch_size = 1  # one sentence
sequence_length = 1 # Let's do one by one
num_layers = 1  # one-layer rnn

# 1. Model

In [89]:
class MyRNN(nn.Module):
    def __init__(self, input_size, hidden_size, sequence_length):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.sequence_length = sequence_length
        
        self.b = torch.zeros(batch_size * sequence_length, batch_size * sequence_length)
        self.W = torch.randn(batch_size * sequence_length, batch_size * sequence_length)
        self.U = torch.randn(batch_size * sequence_length, batch_size * sequence_length)
        self.c = torch.zeros(batch_size * sequence_length, batch_size * sequence_length)
        self.V = torch.randn(batch_size * sequence_length, batch_size * sequence_length)
        
        self.tanh = torch.nn.Tanh()
        self.softmax = torch.nn.Softmax()
        
    def forward(self, input, hidden):
        """
        input.size() = (1, 6, 5) = (batch_size, sequence_length, dimension)
        x = (6, 5) = (batch_size * sequence_length, input_size)
        U = (6, 6) = (batch_size * sequence_length, batch_size * sequence_length)
        h = (6, 6) = (batch_size * sequence_length, batch_size * sequence_length)
        W = (6, 6) = (batch_size * sequence_length, batch _size * sequence_length)
        b = (6, 6) = (batch_size * sequence_length, batch_size * sequence_length)
        hidden = (1, 1, 5)
        """
        
        x = input.view(-1, input_size) # Transpose in here already
        import ipdb; ipdb.set_trace()
        a_trans = self.b + self.W * hidden + torch.mm(self.U, x)
        hidden_trans = self.tanh(a_trans)
        output_trans = self.c + self.V * hidden_trans
        y_hat_trans = self.softmax(output_trans)
                
        # Do not return a transposed value
        return torch.transpose(y_hat_trans, 0 ,1) , torch.transpose(hidden_trans, 0 ,1)

In [90]:
class Model(nn.Module):
    def __init__(self,
                input_size=5,
                hidden_size=5,
                num_layers=1,
                batch_size=1,
                sequence_length=1,
                num_classes=5):
        super().__init__()
        self.rnn = MyRNN(input_size=input_size,
                         hidden_size=hidden_size,
                         sequence_length=sequence_length)
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.sequence_length = sequence_length
        self.num_classes = num_classes
        
        # Fully-Connected layer
        self.fc = nn.Linear(num_classes, num_classes)

    def forward(self, x, hidden):
        # Reshape input in (batch_size, sequence_length, input_size)
        x = x.view(self.batch_size, self.sequence_length, self.input_size)

        out, hidden = self.rnn(x, hidden)
        out = self.fc(out) # Add here
        out = out.view(-1, self.num_classes)
        return hidden, out
    
    def init_hidden(self):
        return torch.zeros(self.num_layers, self.batch_size, self.hidden_size)


# 2. Criterion & Loss

In [91]:
model = Model(sequence_length=6)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

# 3. Training

In [92]:
model = Model(input_size=5, hidden_size=5, num_layers=1, 
              batch_size=1, sequence_length=6, num_classes=5)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

In [93]:
hidden = model.init_hidden()
loss = 0

In [95]:
idx = 0
for epoch in range(0, 10 + 1):
    hidden.detach_()
    hidden = hidden.detach()
    hidden = hidden.clone().detach().requires_grad_(True) # New syntax from `1.0`
    
    hidden, outputs = model(inputs, hidden)
    optimizer.zero_grad()
    loss = criterion(outputs, labels) # It wraps for-loop in here
    loss.backward()
    optimizer.step()
    _, idx = outputs.max(1)
    idx = idx.data.numpy()
    
    # A bit acrobatic since I lookup `value` to see the `key`
    result_str = [list(word_to_ix.keys())[list(word_to_ix.values()).index(i)] for i in idx]
    print(f"epoch: {epoch}, loss: {loss.data}")
    print(f"Predicted string: {''.join(result_str)}")

> <ipython-input-89-05bb374b4076>(28)forward()
     27         import ipdb; ipdb.set_trace()
---> 28         a_trans = self.b + self.W * hidden + self.U * x
     29         hidden_trans = self.tanh(a_trans)

ipdb> self.U
tensor([[ 0.3681, -0.3416, -0.1443, -0.4041, -0.7753,  1.3913],
        [ 0.5287, -0.7498,  0.0993,  0.8978, -0.1096, -1.9167],
        [-0.3768, -1.4041,  0.1669, -1.3869,  1.2175,  0.0552],
        [ 1.0390,  0.8600,  0.7279,  1.0043,  0.6374,  0.1194],
        [-0.0624,  1.5815, -1.3528,  0.8663,  2.1381,  0.4147],
        [-0.0152, -0.2596, -0.3542, -1.1579,  0.4185, -0.0612]])
ipdb> torch.mm(self.U, x)
tensor([[-1.2289e+00, -1.5714e-01, -3.3829e-01, -5.1315e-01, -2.6088e-01],
        [ 8.4075e-01,  2.2220e+00, -1.1178e+00, -8.5992e-01, -1.4519e+00],
        [-2.6236e+00, -2.6904e-03, -2.1259e+00,  6.5612e-01, -2.2658e-01],
        [-1.5007e+00, -7.2256e-01,  2.6194e+00, -4.5754e+00, -1.1471e+00],
        [ 6.4270e-01, -3.4616e+00,  3.4493e+00,  1.3070e+00,  2.7934

BdbQuit: 

In [83]:
rnn = nn.RNN(10, 20, 2)
input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
output, hn = rnn(input, h0)